#### Импорт необходимых модулей

In [8]:
import sqlite3
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score

#### Подключение к базе данных

Получим из базы данных положительные и отрицательные отзывы, сформируем из них два списка.

In [9]:
conn = sqlite3.connect('reviews.db')
cur = conn.cursor()

cur.execute("SELECT text FROM reviews WHERE polarity='pos'")
goodReviews = [review[0] for review in cur.fetchall()]
cur.execute("SELECT text FROM reviews WHERE polarity='neg'")
badReviews = [review[0] for review in cur.fetchall()]

#### Препроцессинг текста

Токенизируем слова, приведём к нижнему регистру и к начальной форме. 

In [10]:
stop_words = set(stopwords.words('english'))
lemmmatizer=WordNetLemmatizer()

def preprocess(reviews):
    tokenized = [nltk.tokenize.word_tokenize(x.lower()) for x in reviews]
    cleaned = [lemmmatizer.lemmatize(i) for j in tokenized for i in j if i.isalpha() and i not in stop_words]
    return cleaned

goodReviewsWords = preprocess(goodReviews)
badReviewsWords = preprocess(badReviews)

#### Построение "модели"

Сформируем два множества - одно будет состоять из слов, встретившихся только в положительных отзывах, а второе - из слов, встретившихся только в отрицательных.

In [11]:
good_words = set(goodReviewsWords).difference(set(badReviewsWords))
bad_words = set(badReviewsWords).difference(set(goodReviewsWords))

Получим из базы данных отзывы на фильм Марсианин, полярность которых будем пытаться угадывать.

In [12]:
cur.execute('SELECT text FROM test')
testReviews = [[review[0]] for review in cur.fetchall()]

Предсказываем, положительный или отрицательный отзыв перед нами в зависимости от того, какие слова в нём встретились: если в нём больше слов из "положительного" множества, говорим, что он положительный, и наоборот.

In [13]:
predictedScores = []
for review in testReviews:
    check = set(preprocess(review))
    if len(check.intersection(good_words)) >= len(check.intersection(bad_words)):
        print(review, 'is claimed to be positive')
        predictedScores.append('pos')
    elif len(check.intersection(good_words)) < len(check.intersection(bad_words)):
        print(review, 'is claimed to be negative')
        predictedScores.append('neg')

["Seeing that the movie was over 2 hrs long, and knowing that the person was going to be stuck on the planet by their self for the majority, I didn't have high hopes for it. Usually these types of movies are boring. Not a lot of writers can pull this off. This movie was really good tho. A group of us watched it and really enjoyed. There were some things they failed to explain, that we collectively came up with our own answer for. Other than that, it was great. I never felt bored or left waiting for more. Overall, I feel it was very well written, produced and acted. Well worth the watching. Some people on here have griped that it was 'predictable', but then, most movies are. Who ever watched Star Wars with the thought that Vader would ultimately win??? Who ever watched Star Trek thinking that Kirk wouldn't find a way to save the day? And frankly, who grew up watching Scooby Doo thinking that someone would actually kill Scooby, Shaggy, Fred, Daphne or Velma???"] is claimed to be negative

#### Оценка точности

Сравним эмпирические данные с фактическими и оценим точность предсказания.

In [14]:
cur.execute('SELECT polarity FROM test')
actualScores = [score[0] for score in cur.fetchall()]

print('Accuracy: %.4f' % accuracy_score(predictedScores, actualScores))

Accuracy: 0.7815


Точность предсказания 78,15%, что кажется довольно неплохим результатом. Улучшить программу можно следующими способами: учитывать частотность слов, а не только факт их наличия в том или ином множестве. Это позволит исключить неоднозначность в случае, когда число "хороших" и "плохих" слов одинаково, и отдать предпочтение слову, имеющему большую значимость. Также думаю, что увеличить точность оценки могло бы сравнение не просто слов по отдельности, но целых составляющих. Ещё одним улучшением может послужить не удаление пунктуации, как мы это делали здесь, а наоборот её анализ. На это обращается внимание во многих исследованиях по сентиментному анализу, например в [Parlar, Özel, Song 2019: 124]: «In the topical classification of text, stemming as well as the removal of stop words and punctuation marks are usually applied to reduce the feature size and improve the classification accuracy. However, punctuation marks and stop words may be important in sentiment analysis, as they can be used to express sentiments.».